In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam, SGD, Adagrad

Using TensorFlow backend.


In [10]:
train_df = pd.read_csv("./data/training2.csv")

In [11]:
# Features and labels
df_feature = train_df.drop(['loan_default'],axis=1)
df_label = train_df['loan_default']
df_feature1 = df_feature

In [12]:
# Feature Imporance using RF-Classifer
clf = RandomForestClassifier()
model = clf.fit(df_feature1, df_label)


feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = df_feature1.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

# top-30 features
df_feature_imp = feature_importances.index[:30]
feaures = df_feature1[df_feature_imp]


/home/ketaki/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [13]:
train_x, test_x, train_y, test_y = train_test_split(feaures, df_label,test_size=0.1)

In [14]:
scaler = StandardScaler()
scaled_train_x = scaler.fit_transform(train_x)
scaled_test_x = scaler.transform(test_x)

In [15]:
model = Sequential()
model.add(Dense(400,input_dim=30))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.2))

# model.add(Dense(100))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(Dropout(0.25))

model.add(Dense(25))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.2))

# model.add(Dense(75))
# model.add(Activation('tanh'))
# model.add(Dropout(0.25))

# model.add(Dense(10))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# # model.add(Dropout(0.25))

model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

# adam = Adagrad(lr=0.001)
# sgd = SGD(lr=0.0001,momentum=0.9, nesterov=True)
model.compile(loss ='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(scaled_train_x, train_y, batch_size=128, epochs=10,verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
209838/209838 [==============================] - 14s 67us/step - loss: 0.5821 - acc: 0.7135
Epoch 2/10
209838/209838 [==============================] - 11s 54us/step - loss: 0.5149 - acc: 0.7830
Epoch 3/10
209838/209838 [==============================] - 11s 55us/step - loss: 0.5134 - acc: 0.7830
Epoch 4/10
209838/209838 [==============================] - 12s 57us/step - loss: 0.5128 - acc: 0.7830
Epoch 5/10
209838/209838 [==============================] - 12s 57us/step - loss: 0.5126 - acc: 0.7830
Epoch 6/10
209838/209838 [==============================] - 11s 55us/step - loss: 0.5124 - acc: 0.7830
Epoch 7/10
209838/209838 [==============================] - 11s 55us/step - loss: 0.5120 - acc: 0.7830
Epoch 8/10
209838/209838 [==================

In [16]:
# scaled_test = scaler.transform(test_x)
pred = model.predict(scaled_test_x)
print(roc_auc_score(test_y, pred))

0.6039691226263205


In [17]:
# AdaBoostClassifier 
clf = AdaBoostClassifier(n_estimators=80, learning_rate=2)
model_ = clf.fit(scaled_train_x, train_y)

pred = model_.predict(scaled_test_x)

roc_auc_score(test_y, pred)

0.4466927986810111

In [18]:
testing_df = pd.read_csv("./data/testing2.csv")

In [20]:
testing_df.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Employment.Type', 'State_ID',
       'Employee_code_ID', 'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag',
       'VoterID_flag', 'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES', 'Age',
       'DisbursalDate'],
      dtype='object')

In [24]:
# Prepare test set
testing_df = pd.read_csv("./data/testing2.csv")

testing_df_ = scaler.transform(testing_df)
testing_df = testing_df[df_feature_imp]

test_pred = model.predict(testing_df)

testing_df = pd.read_csv("./data/sample_submission.csv")
testing_df = testing_df.drop(['loan_default'],axis=1)
testing_df['loan_default']= test_pred

testing_df.to_csv("./data/submission_IMP30_NN_NORM.csv",index=False)

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment.Type,State_ID,Employee_code_ID,...,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,Age,DisbursalDate
0,53478,63558,86.54,67,22807,45,1497,1,6,1998,...,0,0,0,0,0,0,0,0,45,5
1,55513,63163,89.45,67,22807,45,1497,0,6,1998,...,0,5605,0,1,0,8,12,1,34,5
2,65282,84320,79.93,78,23135,86,2071,1,4,1646,...,0,0,0,0,0,0,0,0,24,5
3,46905,63896,76.58,78,17014,45,2070,0,4,1646,...,0,0,0,0,0,29,29,0,46,5
4,51428,63896,86.08,78,17014,45,2069,1,4,1646,...,0,0,0,0,0,0,0,0,47,5
